# Hyperedges Generation

We are preprocessing the tissue into patches to use in inference time

In [12]:
import argparse
import torch
import numpy as np
import os
import sys
from tqdm import tqdm
from torch_geometric.loader import DataLoader
from sklearn.metrics import roc_auc_score, accuracy_score

import_dir = '..'

sys.path.insert(0, import_dir + '/src/model/')
from hypergraph_scattering import HypergraphScatteringNet

sys.path.insert(0, import_dir + '/src/utils/')
from seed import seed_everything
from log_utils import log
from data_utils import split_dataset
from scheduler import LinearWarmupCosineAnnealingLR

sys.path.insert(0, import_dir + '/src/dataset/')
from placenta import PlacentaDatasetHypergraph
from mibi import MIBIDatasetHypergraph
from extend import ExtendedDataset

In [13]:
def prepare_dataloaders(args):
    if args.dataset == 'placenta':
        dataset = PlacentaDatasetHypergraph(data_folder=args.data_folder, k_hop=args.k_hop)
    elif args.dataset == 'mibi':
        dataset = MIBIDatasetHypergraph(data_folder=args.data_folder, k_hop=args.k_hop)

    # Train/val/test split
    ratios = [float(c) for c in args.train_val_test_ratio.split(':')]
    ratios = tuple([c / sum(ratios) for c in ratios])

    train_set, val_set, test_set = split_dataset(
        dataset=dataset,
        splits=ratios,
        random_seed=0)  # Fix the dataset.

    min_batch_per_epoch = 5
    desired_len = args.batch_size * min_batch_per_epoch
    if len(dataset) < desired_len:
        train_set = ExtendedDataset(dataset=train_set, desired_len=desired_len)

    train_loader = DataLoader(train_set, batch_size=args.batch_size, shuffle=True)
    val_loader = DataLoader(val_set, batch_size=args.batch_size, shuffle=False)
    test_loader = DataLoader(test_set, batch_size=args.batch_size, shuffle=False)
    return train_loader, val_loader, test_loader, dataset

In [24]:
from types import SimpleNamespace
args = SimpleNamespace(
    train_val_test_ratio= '6:2:2',
    max_epochs= 50,
    max_training_iters= 64,
    max_validation_iters= 64,
    batch_size= 128,
    desired_batch_size= 256,
    learning_rate= 0.001,
    trainable_scales= True,
    k_hop= 1,
    num_workers= 8,
    random_seed= 1,
    dataset= 'mibi',
    data_folder= '../data/MIBI/patchified_all_genes/',
    num_features= 29
)
print(args.dataset)

prepare_dataloaders(args)

mibi


(<torch_geometric.loader.dataloader.DataLoader at 0x306882520>,
 <mibi.MIBIDatasetHypergraph at 0x306882a90>)